In [1]:
import shutil
import os

dataset_path = "D:\\Job\\image_retrieval\\data\\COVID-19_Radiography_Dataset"
structured_dataset = "D:\\Job\\image_retrieval\\data\\COVID_Dataset"

In [2]:
from dataset import MedicalDataset, get_transforms
from torch.utils.data import DataLoader, random_split

dataset_root = "D:\\Job\\image_retrieval\\data\\COVID_Dataset"

full_dataset = MedicalDataset(dataset_root, transform=get_transforms())

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [5]:
from config_cnn import Config
from cnn_fpn_rmac import CNN_FPN_RMAC
import torch
import torch.nn as nn

cfg = Config()
cfg.NUM_CLASSES = 4  

model = CNN_FPN_RMAC(cfg)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

In [4]:
pip show torch

Name: torch
Version: 2.6.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: d:\app\anaconda3\envs\mir_py310\lib\site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: torchaudio, torchvision
Note: you may need to restart the kernel to use updated packages.


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(25):
    model.train()
    train_loss, train_correct = 0.0, 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x.size(0)
        train_correct += (outputs.argmax(1) == y).sum().item()

    train_acc = train_correct / len(train_dataset)
    train_loss /= len(train_dataset)

    model.eval()
    val_loss, val_correct = 0.0, 0

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)

            val_loss += loss.item() * x.size(0)
            val_correct += (outputs.argmax(1) == y).sum().item()

    val_acc = val_correct / len(val_dataset)
    val_loss /= len(val_dataset)

    print(f"Epoch {epoch+1}/25")
    print(f"Train     Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
    print(f"Val       Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")


Epoch 1/25
Train     Loss: 0.0000 | Acc: 1.0000
Val       Loss: 0.3010 | Acc: 0.9511
Epoch 2/25
Train     Loss: 0.0000 | Acc: 1.0000
Val       Loss: 0.2975 | Acc: 0.9511
Epoch 3/25
Train     Loss: 0.0000 | Acc: 1.0000
Val       Loss: 0.3019 | Acc: 0.9511
Epoch 4/25
Train     Loss: 0.0000 | Acc: 1.0000
Val       Loss: 0.3079 | Acc: 0.9499
Epoch 5/25
Train     Loss: 0.0000 | Acc: 1.0000
Val       Loss: 0.3161 | Acc: 0.9502
Epoch 6/25
Train     Loss: 0.0000 | Acc: 1.0000
Val       Loss: 0.3182 | Acc: 0.9511
Epoch 7/25
Train     Loss: 0.0000 | Acc: 1.0000
Val       Loss: 0.3173 | Acc: 0.9504
Epoch 8/25
Train     Loss: 0.0000 | Acc: 1.0000
Val       Loss: 0.3672 | Acc: 0.9435
Epoch 9/25
Train     Loss: 0.0817 | Acc: 0.9712
Val       Loss: 0.2409 | Acc: 0.9381
Epoch 10/25
Train     Loss: 0.0082 | Acc: 0.9977
Val       Loss: 0.2413 | Acc: 0.9421
Epoch 11/25
Train     Loss: 0.0018 | Acc: 0.9998
Val       Loss: 0.2446 | Acc: 0.9490
Epoch 12/25
Train     Loss: 0.0005 | Acc: 1.0000
Val       Loss

In [12]:
#save model
model_save_path = "D:\\Job\\image_retrieval\\models\\covid_cnn_fpn_rmac.pth"
torch.save(model.state_dict(), model_save_path)